In [1]:
import copy
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import pandas as pd
import logging
from torch.utils.data import Dataset
from torch.utils.data import DataLoader


In [2]:
log = logging.getLogger("test_logger")

In [5]:
class RNN(nn.Module):
    """ 
    The (one to one) RNN class.
    """
    def __init__(self, input_size: int, hidden_size: int, output_size: int,bias: bool = False, activation1: str = "sigmoid", activation2: str = "sigmoid") -> None:
        """ 
        Constructor of the class.
        Parameters
        ----------
        input_size: embedding size of the input
        hidden_size: hidden size
        output_size: embedding size of the output
        bias: set to "True" to enable bias
        activation1: set the activation to calc the a<t>
        activation2: set the activation to calc the y<t>
        Returns
        -------
        Nothing.
        """
        if any(x < 0 for x in [input_size, hidden_size, output_size]):
            log.error("Negative number detected in the parameters. Please review and re-init the object.")
        if type(bias)!= bool:
            log.error("Bias parameter is not bool variable. Please review and re-init the object.")
        super(RNN, self).__init__()
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.aa = nn.Linear(hidden_size, hidden_size)
        self.ax = nn.Linear(input_size, hidden_size)
        self.ya = nn.Linear(hidden_size, output_size)
        self.a = torch.zeros(1, hidden_size)
        self.bias = bias
        if bias:
            self.ba = torch.rand(1,hidden_size)
            self.by = torch.rand(1, output_size)
        self.activation1 = self.define_activation(activation1)
        self.activation2 = self.define_activation(activation2)
        
    def define_activation(self, typeActivation: str) -> None:
        """
        Define the activation function for this class
        Parameters
        ----------
        typeActivation: type of activation function
        Returns
        -------
        Nothing
        """
        activation1: str = typeActivation
        if activation1 == "sigmoid":
            return nn.Sigmoid()
        elif activation1 == "tanh":
            return nn.Tanh()
        elif activation1 == "relu":
            return nn.ReLU()
        else:
            log.error("Wrong type of activation. Change it to sigmoid...")
            return nn.Sigmoid()
        
        
    def forward(self, x: torch.Tensor)-> torch.Tensor:
        """ 
        Forward function of the class.
        Parameters
        ----------
        x: The input
        Returns
        -------
        y_t: The result of RNN.
        """
#         print("X shape: ", x.clone().detach().shape)
        if x.clone().detach().shape[1] != self.input_size:
            log.error("Wrong input size!")
            return
        if not self.bias:
            a_t = self.activation1(self.aa(self.a) + self.ax(x))
            self.a = a_t
            y_t = self.activation2(self.ya(a_t))
        else:
            a_t = self.activation1(self.aa(self.axa) + self.ax(x)+self.ba)
            self.a = a_t
            y_t = self.activation2(self.ya(a_t) + self.by)
#         print(a_t.shape)
        return y_t
    
    def reset_a(self) -> None:
        """
        Reset self.a to avoid error.
        Parameters
        ----------
        Nothing
        Returns
        -------
        Nothing
        """
        self.a = torch.zeros(1, self.hidden_size)


In [6]:
test= RNN(32, 512, 256)
test.forward(torch.rand(64,32))

tensor([[0.3576, 0.5170, 0.4495,  ..., 0.5280, 0.4868, 0.4725],
        [0.3710, 0.5326, 0.4365,  ..., 0.5375, 0.4780, 0.4809],
        [0.3598, 0.5186, 0.4480,  ..., 0.5276, 0.4934, 0.4786],
        ...,
        [0.3693, 0.5250, 0.4363,  ..., 0.5294, 0.4921, 0.4784],
        [0.3665, 0.5202, 0.4416,  ..., 0.5306, 0.4858, 0.4691],
        [0.3590, 0.5239, 0.4417,  ..., 0.5240, 0.4826, 0.4725]],
       grad_fn=<SigmoidBackward0>)

In [10]:
import random 
w = random.random()
w

0.3539535256996288

In [7]:
type(test)

__main__.RNN

In [350]:
testt= torch.rand(64,128,32)

In [351]:
testt[:,:,0].shape

torch.Size([64, 128])

In [352]:
class manyToOneRNN(nn.Module):
    """
    Many to one RNN class.
    """
    def __init__(self, input_times: int, input_size: int, hidden_size: int, output_size: int, bias: bool = False, activation1: str = "sigmoid", activation: str = "sigmoid") -> None:
        """
        Constructor of the class.
        Parameters
        ----------
        input_times: Times of inputs
        other parameters: Same with RNN class
        Returns
        -------
        Nothing
        """
        if input_times < 1: 
            log.error("Input times less than 1. This object stills be created but won't work!")
        super(manyToOneRNN, self).__init__()
        self.rnn = RNN(input_size, hidden_size, output_size, bias, activation1, activation2)
        self.input_times = input_times
    def forward(self, x: torch.Tensor) -> torch.Tensor:
        """
        Forward function of this class
        Parameters
        ----------
        x: Input
        Returns
        -------
        Nothing
        """
        if x.clone().detach().shape[1] != self.input_times:
            log.error("Wrong input size!")
            return
        for i in range(x.shape[1]):
            y_t = self.rnn.forward(x[:,i,:])
        self.reset_a()
        return y_t
    
    def reset_a(self) -> None:
        """
        Reset self.a of self.rnn to avoid error.
        Parameters
        ----------
        Nothing
        Returns
        -------
        Nothing
        """
        self.rnn.reset_a()

In [353]:
class oneToManyRNN(nn.Module):
    """
    The one to many RNN class.
    """
    def __init__(self, output_times: int, input_size: int, hidden_size: int, output_size: int, bias: bool = False, activation1: str = "sigmoid", activation2: str = "sigmoid") -> None:
        """
        Constructor of the class.
        Parameters
        ----------
        output_times: Times of outputs
        other parameters: Same with RNN class
        Returns
        -------
        Nothing
        """
        if output_times < 1:
            log.error("Output times < 1. It will cause errors in the future. Please re-init the object.")
        if input_size != output_size:
            log.error("Input size and output size is different. This object stills be created but won't work!")
        super(oneToManyRNN, self).__init__()
        self.rnn = RNN(input_size, hidden_size, output_size, bias, activation1, activation2)
        self.output_times = output_times
        
    def forward(self, x: torch.Tensor) -> torch.Tensor:
        """
        Forward function of this class
        Parameters
        ----------
        x: Input
        Returns
        -------
        Nothing
        """
        if len(x.clone().detach().shape) != 3:
            log.error("Wrong input size!")
            return
        result = torch.Tensor([])
        y_t = self.rnn.forward(x)
        result = torch.cat((result, y_t),0)
#         print(result)
        for i in range(self.output_times - 1):
            y_t = self.rnn.forward(y_t)
            result = torch.cat((result,y_t),0)
#             print(y_t)
        self.reset_a()
        return torch.reshape(result,(x.shape[0], self.output_times, self.rnn.output_size)) # Batch size, output times, output size
    
    def reset_a(self) -> None:
        """
        Reset self.a of self.rnn to avoid error.
        Parameters
        ----------
        Nothing
        Returns
        -------
        Nothing
        """
        self.rnn.reset_a()

In [364]:
class manyToManyRNN(nn.Module):
    """
    Many to many RNN class.
    """
    def __init__(self, input_times: int, output_times: int, input_size: int, hidden_size: int, output_size: int, bias: bool = False, activation1: str = "sigmoid", activation2: str = "sigmoid", simultaneous: bool = False) -> None:
        """
        Constructor of the class
        Parameters
        ----------
        input_times: Times of the input
        output_times: Times of the output
        simultaneous: Choose if RNN receive all the inputs before 
        other parameters: Same with RNN class.
        Returns
        -------
        Nothing
        """
        if output_times < 1 or input_times < 1:
            log.error("Either input times or output times < 1. It will cause errors in the future. Please re-init the object.")
        if input_size != output_size:
            log.error("Input size and output size is different. This object stills be created but won't work!")
        super(manyToManyRNN, self).__init__()
        self.rnn = RNN(input_size, hidden_size, output_size, bias, activation1, activation2)
        self.input_times = input_times
        self.output_times = output_times
        self.simultaneous = simultaneous
        
    def forward(self, x: torch.Tensor) -> torch.Tensor:
        """
        Forward function of this class
        Parameters
        ----------
        x: Input
        Returns
        -------
        Nothing
        """
        if len(x.shape) != 3:
            log.error("Wrong input size!")
        result = torch.tensor([])
        if self.simultaneous:
            for i in range(x.shape[1]):

                y_t = self.rnn.forward(x[:,i,:])
                result = torch.cat((result, y_t),0)
        else:
            y_t = 0
            for i in range(x.shape[1]):
      
                y_t = self.rnn.forward(x[:,i,:])

            for i in range(self.output_times):
           
                y_t = self.rnn.forward(y_t)
                result = torch.cat((result, y_t),0)
        self.reset_a()
        return torch.reshape(result,(x.shape[0], self.output_times, self.rnn.output_size)) # Batch size, output times, output size
    
    def reset_a(self) -> None:
        """
        Reset self.a of self.rnn to avoid error.
        Parameters
        ----------
        Nothing
        Returns
        -------
        Nothing
        """
        self.rnn.reset_a()

In [365]:
x_test = torch.reshape(torch.Tensor([1.,2.,3.,4.]),(2,2,1))
y_test = torch.reshape(torch.Tensor([.1, .1, .2, .2, .3, .3, .4, .4, .2, .2, .3, .3, .4, .4, .5, .5]), (2,8,1))

In [366]:
y_test.shape

torch.Size([2, 8, 1])

In [378]:
model = manyToManyRNN(2, 8, 1, 16, 1, simultaneous= False)

In [379]:
x = model.forward(x_test)
print(x)
print(x.shape)

tensor([[[0.4960],
         [0.4910],
         [0.5023],
         [0.5020],
         [0.5023],
         [0.5023],
         [0.5023],
         [0.5023]],

        [[0.5023],
         [0.5023],
         [0.5023],
         [0.5023],
         [0.5023],
         [0.5023],
         [0.5023],
         [0.5023]]], grad_fn=<ReshapeAliasBackward0>)
torch.Size([2, 8, 1])


In [380]:
model.parameters()

<generator object Module.parameters at 0x79c70deb0430>

In [381]:
loss_fn = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.2)


In [382]:
model.train()

manyToManyRNN(
  (rnn): RNN(
    (aa): Linear(in_features=16, out_features=16, bias=True)
    (ax): Linear(in_features=1, out_features=16, bias=True)
    (ya): Linear(in_features=16, out_features=1, bias=True)
    (activation1): Sigmoid()
    (activation2): Sigmoid()
  )
)

In [389]:
for i in range(5):
#     model.hidden = model.init_hidden()
    

    
    y_pred = model.forward(x_test)
    print(y_pred)
    optimizer.zero_grad()
    loss = loss_fn(y_pred, y_test)

    
    loss.backward()
    optimizer.step()


tensor([[[0.0999],
         [0.1001],
         [0.2004],
         [0.1997],
         [0.2991],
         [0.2991],
         [0.4000],
         [0.4000]],

        [[0.3004],
         [0.3004],
         [0.4000],
         [0.4000],
         [0.3004],
         [0.3004],
         [0.4000],
         [0.4000]]], grad_fn=<ReshapeAliasBackward0>)
tensor([[[0.0999],
         [0.1001],
         [0.2004],
         [0.1997],
         [0.2991],
         [0.2991],
         [0.4000],
         [0.4000]],

        [[0.3004],
         [0.3004],
         [0.4000],
         [0.4000],
         [0.3004],
         [0.3004],
         [0.4000],
         [0.4000]]], grad_fn=<ReshapeAliasBackward0>)
tensor([[[0.0999],
         [0.1001],
         [0.2004],
         [0.1997],
         [0.2991],
         [0.2991],
         [0.4000],
         [0.4000]],

        [[0.3004],
         [0.3004],
         [0.4000],
         [0.4000],
         [0.3004],
         [0.3004],
         [0.4000],
         [0.4000]]], grad_fn=<R

In [373]:
y_pred = model.forward(x_test)
y_pred

tensor([[[0.0820],
         [0.0574],
         [0.2550],
         [0.2404],
         [0.3045],
         [0.2872],
         [0.3516],
         [0.3474]],

        [[0.3584],
         [0.3582],
         [0.3577],
         [0.3583],
         [0.3554],
         [0.3556],
         [0.3551],
         [0.3552]]], grad_fn=<ReshapeAliasBackward0>)

# View dataset

In [3]:
df = pd.read_csv("/kaggle/input/stock-exchange-data/indexData.csv")

In [55]:
df["Year"]= pd.to_datetime(df['Date']).dt.year
df["Month"] = pd.to_datetime(df['Date']).dt.month
df["Day"] = pd.to_datetime(df['Date']).dt.day

In [62]:
df.iloc[:,2:11]

,Open,High,Low,Close,Adj Close,Volume,Year,Month,Day
0,528.690002,528.690002,528.690002,528.690002,528.690002,0.0,1965,12,31
1,527.210022,527.210022,527.210022,527.210022,527.210022,0.0,1966,1,3
2,527.840027,527.840027,527.840027,527.840027,527.840027,0.0,1966,1,4
3,531.119995,531.119995,531.119995,531.119995,531.119995,0.0,1966,1,5
4,532.070007,532.070007,532.070007,532.070007,532.070007,0.0,1966,1,6
...,...,...,...,...,...,...,...,...,...
112452,1241.119995,1251.910034,1241.119995,1247.069946,1247.069946,379696400.0,2021,5,27
112453,1249.469971,1259.209961,1249.030029,1256.599976,1256.599976,160773400.0,2021,5,28
112454,1256.079956,1258.880005,1248.140015,1248.930054,1248.930054,91173700.0,2021,5,31
112455,1254.609985,1265.660034,1254.609985,1258.579956,1258.579956,155179900.0,2021,6,1


In [75]:
df = df.dropna()
df

,Index,Date,Open,High,Low,Close,Adj Close,Volume,Year,Month,Day
0,NYA,1965-12-31,528.690002,528.690002,528.690002,528.690002,528.690002,0.0,1965,12,31
1,NYA,1966-01-03,527.210022,527.210022,527.210022,527.210022,527.210022,0.0,1966,1,3
2,NYA,1966-01-04,527.840027,527.840027,527.840027,527.840027,527.840027,0.0,1966,1,4
3,NYA,1966-01-05,531.119995,531.119995,531.119995,531.119995,531.119995,0.0,1966,1,5
4,NYA,1966-01-06,532.070007,532.070007,532.070007,532.070007,532.070007,0.0,1966,1,6
...,...,...,...,...,...,...,...,...,...,...,...
112452,N100,2021-05-27,1241.119995,1251.910034,1241.119995,1247.069946,1247.069946,379696400.0,2021,5,27
112453,N100,2021-05-28,1249.469971,1259.209961,1249.030029,1256.599976,1256.599976,160773400.0,2021,5,28
112454,N100,2021-05-31,1256.079956,1258.880005,1248.140015,1248.930054,1248.930054,91173700.0,2021,5,31
112455,N100,2021-06-01,1254.609985,1265.660034,1254.609985,1258.579956,1258.579956,155179900.0,2021,6,1


In [66]:
df_tensor = torch.tensor(df.iloc[:,2:11].drop(columns=["Close", "Adj Close"]).values)
df_tensor.shape

torch.Size([112457, 7])

In [68]:
target_tensor = torch.tensor(df['Close'].values)
target_tensor.shape

torch.Size([112457])

In [ ]:
class CustomStarDataset(Dataset)